In [4]:
%matplotlib inline
from glob import glob

import matplotlib.pyplot as plt
import numpy as np

# Self-diffusion & Conductivity (MSD)

In [12]:
# select what to plot
rho = 1.0753  # 1.1551 # 1.0753
r = f"{rho:.2f}"
T = 340
geos = ["a32b32i0", "a16b16i16", "a0b0i32"]
maxt = 1200
mint = 100

In [13]:
def report_msd(data, pdata, rho):
    # constants
    NA = 6.022e23
    MW = (60.052 + 82.10) * 1e-3
    q = 1.602176634e-19
    T = 300
    kB = 1.380649e-23
    ne_pre = rho * 1e3 / MW * q**2 / (kB * T) * NA * 10 * 1e-10 * 100 / 6
    gk_pre = rho * 1e3 / MW * q**2 / (kB * T) * NA * 10 / 32 * 1e-10 * 100 / 6

    maxs = np.sum(data[0][:, 0] < maxt)
    mins = np.sum(data[0][:, 0] < mint)
    data = np.array(data)
    pdata = np.array(pdata)
    t = data[0, mins:maxs, 0]
    msds = data[:, mins:maxs, 1:]
    pmsd = pdata[:, mins:maxs, 1]
    # D_slopes = (msds[:, -1, :] -msds[:, 0, :]) / (t[-1]- t[0])
    D_slopes = np.array([np.polyfit(t, msd, 1)[0] for msd in msds])
    P_slope = np.array([np.polyfit(t, msd, 1)[0] for msd in pmsd])
    D_pre = 1.0 / 6.0 * 1e2

    fmt = lambda x: f"{np.mean(x):5.2f}({np.std(x):.2f})"
    D_H = f"& {fmt(D_slopes[:,0]*D_pre)}"
    D_O = f"& {fmt(D_slopes[:,1]*D_pre)}"
    D_N = f"& {fmt(D_slopes[:,2]*D_pre)}"
    S_GK = f"& {fmt(P_slope * gk_pre)}"
    S_NE = f"S_NE:{fmt((D_slopes[:,2]+D_slopes[:,1]) * ne_pre )}"
    Delta = f"& {fmt(1-(P_slope * gk_pre)/((D_slopes[:,2]+D_slopes[:,1]) * ne_pre))}"
    print(D_H, D_N, D_O, S_GK, Delta)


for tag in ["pinet2-revpbe", "pinet2-scan"]:
    exp = f"../exp/{tag}"
    cp2k = f"nvt-{tag}"
    ana = f"{exp}/analyses"

    data_cons = [
        np.load(f)
        for f in glob(f"{exp}/const/analyses/*/msd-0-5ns/msd.npy")
    ]
    data_prod = [
        np.load(f"{exp}/analyses/prod/nvt-{T}k-5ns-0/{geo}-r{r}/msd-0-5ns/msd.npy")
        for geo in geos
    ]

    pdata_cons = [
        np.load(f)
        for f in glob(f"{exp}/const/analyses/*/msd-0-5ns/pmsd.npy")
    ]
    pdata_prod = [
        np.load(f"{exp}/analyses/prod/nvt-{T}k-5ns-0/{geo}-r{r}/msd-0-5ns/pmsd.npy")
        for geo in geos
    ]
    print(f"PiNN-{tag}", end="\t")
    report_msd(data_prod, pdata_prod, rho)
    print(f"PiNN-{tag}-c", end="\t")
    report_msd(data_cons, pdata_cons, rho)

PiNN-pinet2-revpbe	&  0.96(0.09) &  1.06(0.07) &  0.97(0.09) &  6.63(1.67) &  0.89(0.02)
PiNN-pinet2-revpbe-c	&  0.82(0.09) &  0.94(0.14) &  0.87(0.08) &  4.58(1.62) &  0.91(0.03)
PiNN-pinet2-scan	&  3.18(0.14) &  3.68(0.11) &  3.08(0.14) & 16.06(7.18) &  0.92(0.04)
PiNN-pinet2-scan-c	&  2.94(0.32) &  3.39(0.31) &  2.95(0.34) &  5.50(3.02) &  0.97(0.02)


In [99]:
from scipy.optimize import curve_fit


def fit(t, prob):
    def f(x, A, t1, t2):
        return A * np.exp(-x / t1) + (1 - A) * np.exp(-x / t2)

    (A, t1, t2), _ = curve_fit(f, t, prob, p0=[0.5,10,1], maxfev=5000)
    # print(A, t1, t2)
    return A * t1 + (1 - A) * t2


pre_scan = "../exp/scan/analyses/cp2k/a*-rho1.0753"
pre_scan_acle_300 = "../exp/scan/analyses/prod/nvt-300k-5ns-0/a*-r1.08/"
pre_scan_acle_340 = "../exp/scan/analyses/prod/nvt-340k-5ns-0/a*-r1.08/"
pre_revpbe = "../exp/hicut/analyses/cp2k/a*-rho1.0753"
pre_revpbe_acle_300 = "../exp/hicut/analyses/prod/nvt-300k-5ns-0/a*-r1.08/"
pre_revpbe_acle_340 = "../exp/hicut/analyses/prod/nvt-340k-5ns-0/a*-r1.08/"


for pre in [
    pre_scan,
    pre_revpbe,
    pre_scan_acle_300,
    pre_revpbe_acle_300,
    pre_scan_acle_340,
    pre_revpbe_acle_340,
]:
    files = glob(f"{pre}/persist.dat")
    t, a, b, p = np.mean([np.loadtxt(f) for f in files], axis=0).T
    print(f"{fit(t, a/a[0]):.2f}, {fit(t, b/b[0]):.2f}, {fit(t, p/p[0]):.2f} ")

12.70, 5.01, 67.68 
22.35, 2.58, 74.69 
11.90, 5.54, 340.83 
17.46, 8.09, 253.12 
8.67, 4.13, 115.68 
12.06, 5.66, 84.10 


In [127]:
k1 = 8.67
k2 = 11.90
T1 = 300
T2 = 340
R = 8.31446261815324

Eact= - (np.log(k1/k2)*R)/(1/T1-1/T2)
print(Eact/1000)

6.713990954047353
